# Initialize

In [1]:
import sys
sys.executable

'C:\\Users\\Adrian\\Miniconda3\\envs\\cocktails\\python.exe'

In [2]:
# Load plugins
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import json

# Set plugin options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)
%pylab inline
pylab.rcParams['figure.figsize'] = (10, 6)

# Set jupyter options
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#InteractiveShell.ast_node_interactivity = "last_expr"

Populating the interactive namespace from numpy and matplotlib


# Scrape web

In [3]:
import requests
import time

## Get Categories

First, get categories of products we wish to scrape from the website homepage

In [4]:
home_url = "https://www.thewhiskyexchange.com/"
#url2 = 'https://www.thewhiskyexchange.com/c/338/gin?filter=true&rfdata=#productlist-filter'
response = requests.get(home_url)
response

<Response [200]>

In [5]:
home_soup = BeautifulSoup(response.text, "html.parser")
# Preview
#home_soup

In [6]:
# Get all of the top menu links
menu_top_links = home_soup.find_all('li', class_='nav-bar__item js--navitem')

# Verify length
print(len(menu_top_links))

11


Of the 11 main website menu links, we only want to access specific ones: Whiskey, Spirits & Liqueurs, etc.

In [7]:
# Get Spirits

# Store only 6th element for now, as we know that is for the category "Spirits & Liqueurs"
sl = menu_top_links[5]

# Keep only the portion of the header that we are interested in
spirits = sl.find('h3', text='Shop by Spirit\n').parent.parent
liqueurs = sl.find('h3', text='Liqueurs\n').parent.findNext('div')

# Combine spirits and liqueurs
spirits_liqueurs = str(spirits) + str(liqueurs)
spirits_liqueurs = BeautifulSoup(spirits_liqueurs, "html.parser")

# Get all categories and children (if any)
spirits_liqueurs_cat = spirits_liqueurs.find_all(lambda tag: tag.name == 'ul' and tag.get('class') == ['nav-panel__list'])

In [8]:
# Get Whiskey

# Store only 6th element for now, as we know that is for the category "Spirits & Liqueurs"
whiskey = menu_top_links[4]

# Keep only the portion of the header that we are interested in
whiskey = whiskey.find('h3', text='Shop by Type\n').parent.parent

# Get all categories and children (if any)
whiskey_cat = whiskey.find_all(lambda tag: tag.name == 'ul' and tag.get('class') == ['nav-panel__list'])

In [9]:
# Get Scotch

# Store only 6th element for now, as we know that is for the category "Spirits & Liqueurs"
scotch = menu_top_links[3]

# Keep only the portion of the header that we are interested in
scotch = scotch.find('h3', text='Shop by Type\n').parent.parent

# Get all categories and children (if any)
scotch_cat = scotch.find_all(lambda tag: tag.name == 'ul' and tag.get('class') == ['nav-panel__list'])

Define function to loop through each menu link and extract all relevant categories and subcategories.
- Ex: Menu link --> "Spirits and Liqueurs"
- Category: "Cognac"
- Subcategory: "XO Cognac"

In [10]:
def get_categories(category_tags):
    categories = []
    for i, cat in enumerate(category_tags):
        # Create temporary dictionary to category info
        cat_dict = {}

        # Get category info
        cat_dict['category_name'] = cat.find(lambda tag: tag.name == 'a' and tag.get('class') == ['nav-link']).text.strip()
        cat_dict['category_link'] = cat.find(lambda tag: tag.name == 'a' and tag.get('class') == ['nav-link']).get('href')

        # Get subcategory info (if it exists)
        sl_subcat = cat.find_all('a', class_='nav-link--sub-link')
        subcat_list = []

        for j, subcat in enumerate(sl_subcat):
            subcat_dict = {}
            subcat_dict['subcategory_name'] = subcat.text.strip()
            subcat_dict['subcategory_link'] = subcat.get('href')
            subcat_list.append(subcat_dict)

        cat_dict['subcategories'] = subcat_list

        # Append dictionary to df
        categories.append(cat_dict)
    return categories

In [11]:
categories_master = []
categories_master.append(get_categories(spirits_liqueurs_cat))
categories_master.append(get_categories(whiskey_cat))
categories_master.append(get_categories(scotch_cat))

In [13]:
# Flatten list by 1 level to remove the "menu level" that is no longer needed
categories_master_flat = []
for sublist in categories_master:
    for item in sublist:
        categories_master_flat.append(item)

In [16]:
# Preview
print(json.dumps([x for x in categories_master_flat[0:3]], sort_keys=False, indent=4, ensure_ascii=False))

[
    {
        "category_name": "Cognac",
        "category_link": "/c/351/cognac",
        "subcategories": [
            {
                "subcategory_name": "VS",
                "subcategory_link": "/c/352/vs-cognac"
            },
            {
                "subcategory_name": "VSOP",
                "subcategory_link": "/c/354/vsop-cognac"
            },
            {
                "subcategory_name": "XO and Napoléon",
                "subcategory_link": "/c/433/xo-and-napoleon-cognac"
            },
            {
                "subcategory_name": "Special Cuvée",
                "subcategory_link": "/c/432/special-cuvee-cognac"
            },
            {
                "subcategory_name": "Vintage",
                "subcategory_link": "/c/628/vintage-cognac"
            },
            {
                "subcategory_name": "Prestige",
                "subcategory_link": "/c/353/prestige-cognac"
            }
        ]
    },
    {
        "category_name": "Armagnac",

In [15]:
# TODO - Optimize this section
del_list = []

# Create list of items to be deleted
for category in categories_master_flat:
    # Loop through all key value pairs in a category
    for cat_key, cat_value in list(category.items()):
        if cat_value == "Distillery Bottlings" or cat_value == "Independent Bottlings":  
            print(cat_value)
            print(category)
            del_list.append(category)

# Delete items
categories_master_flat = [item for item in categories_master_flat if item not in del_list]

Distillery Bottlings
{'category_name': 'Distillery Bottlings', 'category_link': '/c/40/single-malt-scotch-whisky?filter=true&rfdata=~bottler.Distillery_Bottling', 'subcategories': []}
Independent Bottlings
{'category_name': 'Independent Bottlings', 'category_link': '/c/40/single-malt-scotch-whisky?filter=true&rfdata=~bottler.Independent_Bottling', 'subcategories': []}


In [17]:
# Preview modified list
print(json.dumps([x for x in categories_master_flat[0:3]], sort_keys=False, indent=4, ensure_ascii=False))

[
    {
        "category_name": "Cognac",
        "category_link": "/c/351/cognac",
        "subcategories": [
            {
                "subcategory_name": "VS",
                "subcategory_link": "/c/352/vs-cognac"
            },
            {
                "subcategory_name": "VSOP",
                "subcategory_link": "/c/354/vsop-cognac"
            },
            {
                "subcategory_name": "XO and Napoléon",
                "subcategory_link": "/c/433/xo-and-napoleon-cognac"
            },
            {
                "subcategory_name": "Special Cuvée",
                "subcategory_link": "/c/432/special-cuvee-cognac"
            },
            {
                "subcategory_name": "Vintage",
                "subcategory_link": "/c/628/vintage-cognac"
            },
            {
                "subcategory_name": "Prestige",
                "subcategory_link": "/c/353/prestige-cognac"
            }
        ]
    },
    {
        "category_name": "Armagnac",

## Get bottles

Define function to extract all bottles and description for each category/subcategory in our defined product heirarchy

In [18]:
def get_bottles(url_base):
    # Request page
    product_url = "https://www.thewhiskyexchange.com/"+url_base+'?status=no#productlist-filter'
    cookies = dict(rtwe_paging='pagesize=200', rtwe_viewmode='mode=list')
    response = requests.get(product_url, cookies=cookies)
    # Get page soup
    product_soup = BeautifulSoup(response.text, "html.parser")
    # Get number of pages
    pages = product_soup.find_all('a', class_='page-link')
    if not pages:
        max_pages = 1
    else:
        max_pages = int(pages[len(pages)-2].text)
    # Empty dictionary to store all found products
    products_list = []
    # Loop through all pages and store products + description in dictionary
    for i in range(1, max_pages+1):
        # Convert i to character, as required for string manipulation in the URL below
        pg = str(i)
        # Request page
        product_url = "https://www.thewhiskyexchange.com/"+url_base+'?pg='+pg+'&status=no#productlist-filter'
        cookies = dict(rtwe_paging='pagesize=200', rtwe_viewmode='mode=list')
        response = requests.get(product_url, cookies=cookies)
        # Get page soup
        product_soup = BeautifulSoup(response.text, "html.parser")
        # Find tags for all products
        products_soup = product_soup.find_all('div', class_='information')
        for single_product_soup in products_soup:
            # Create empty dictionary to store product
            product_info={}
            # Get name and description
            product_info['product_name'] = single_product_soup.find('p', class_='name').find(text=True, recursive=False).strip()
            product_info['product_description'] = single_product_soup.find('p', class_='description').text
            # Store in list
            products_list.append(product_info)  
    return products_list

Traverse product heirarchy and populate with extracted bottles

In [19]:
# Loop through each category
for category in categories_master_flat:
    # Loop through all key value pairs in a category
    for cat_key, cat_value in list(category.items()):
        # See if any subcategories exist, if not, just print the CATEGORY url
        if cat_key == "subcategories" and len(cat_value) == 0:            
            listForm = list(category.values())
            temp_dict = {}
            temp_dict['bottles'] = get_bottles(listForm[1])
            cat_value.append(temp_dict)
        # if any subcategories do exist, do not print the CATEGORY url, begin to loop through the list of SUBCATEGORIES
        elif cat_key == "subcategories" and len(cat_value) != 0:
            # Loop through list of categories
            for subcategory in cat_value:
                # Loop through all key value pairs in a subcategory
                for subcat_key, subcat_value in list(subcategory.items()):
                    # Print the SUBCATEGORY url
                    if subcat_key == "subcategory_link":
                        subcategory['bottles'] = get_bottles(subcat_value)
    import time
    time.sleep(2)

In [41]:
# Preview
print(json.dumps([x for x in categories_master_flat], sort_keys=False, indent=4, ensure_ascii=False)[:3000])

[
    {
        "category_name": "Cognac",
        "category_link": "/c/351/cognac",
        "subcategories": [
            {
                "subcategory_name": "VS",
                "subcategory_link": "/c/352/vs-cognac",
                "bottles": [
                    {
                        "product_name": "Hennessy VS Cognac",
                        "product_description": "A 20cl bottle of Hennessy VS. This ever popular entry level cognac from one of the big players in the market, this is ideal for use with a mixer."
                    },
                    {
                        "product_name": "Bache Gabrielsen Tre Kors VS Cognac",
                        "product_description": "Tre Kors means 'three crosses' in Norwegian and the name was carefully chosen – when Norway banned distilled spirits in 1916, Thomas Bache-Gabrielsen, who was Norweigan by birth, added the crosses to symbolise the fight against disease. This was because distilled spirits could still be sold in p

In [27]:
# Export to JSON file for inspection
with open('data.json', 'w') as f:
  json.dump(categories_master_flat, f, sort_keys=False, indent=4, ensure_ascii=False)

Flatten all of this shit (will be needed at some point)

In [48]:
# Create dataframe at category level
df = pd.DataFrame(categories_master_flat)

# Explode subcategories to their own rows, creating a multi-index
df = df.explode('subcategories')

# Convert subcategory dictionary into columns
df = df.set_index(['category_name', 'category_link'])['subcategories'].apply(pd.Series)
df = df.reset_index()

# Explode bottles to their own rows, further expanding the multi-index
df = df.explode('bottles')

# Convert bottles dictionary into separate columns
df = df.set_index(['category_name', 'category_link', 'subcategory_name', 'subcategory_link'])['bottles'].apply(pd.Series)
df = df.reset_index()

# Preview
df.head(20)

,category_name,category_link,subcategory_name,subcategory_link,0,product_description,product_name
0,Cognac,/c/351/cognac,VS,/c/352/vs-cognac,nan,A 20cl bottle of Hennessy VS. This ever popula...,Hennessy VS Cognac
1,Cognac,/c/351/cognac,VS,/c/352/vs-cognac,nan,Tre Kors means 'three crosses' in Norwegian an...,Bache Gabrielsen Tre Kors VS Cognac
2,Cognac,/c/351/cognac,VS,/c/352/vs-cognac,nan,World-famous VS from one of the 'Big Four' pro...,Courvoisier VS Cognac
3,Cognac,/c/351/cognac,VS,/c/352/vs-cognac,nan,One of the most recognisable brands in the wor...,Martell VS Cognac
4,Cognac,/c/351/cognac,VS,/c/352/vs-cognac,nan,Tre Kors means 'three crosses' in Norwegian an...,Bache Gabrielsen VS Cognac
5,Cognac,/c/351/cognac,VS,/c/352/vs-cognac,nan,"A fresh and youthful Cognac, Montuifaud's VS h...",Chateau de Montifaud VS Fine Petite Champagne ...
6,Cognac,/c/351/cognac,VS,/c/352/vs-cognac,nan,"The entry point into the Maxime Trijol, their ...",Maxime Trijol VS Cognac
7,Cognac,/c/351/cognac,VS,/c/352/vs-cognac,nan,Benchmark VS from one of Cognac's 'Big Four' p...,Hennessy VS Cognac
8,Cognac,/c/351/cognac,VS,/c/352/vs-cognac,nan,Matured in virgin Limousin-oak casks for at le...,François Voyer VS Cognac
9,Cognac,/c/351/cognac,VS,/c/352/vs-cognac,nan,An intense yet smooth three-year-old VS Cognac...,Vallein-Tercinier VS Selection Cognac


Write to file

In [28]:
#df.to_csv('output/spirits.csv', index=False, encoding='utf-8-sig')